In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [9]:
import pandas as pd
from datetime import datetime


pos = 0
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,dois_riachos


In [10]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [11]:
def people():
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")

    children = nav.find_element(By.ID, 'maxAge')
    nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
    time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

#     pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
#     time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [12]:
def get_table():
    
    list_links = []
    people()
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                traceback.print_exc()
                
            if NoSuchElementException: 
                traceback.print_exc() 
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [13]:
def get_individual_info():
    
    individual = get_table()
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], []
    
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        linha_dois = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[1]').text
        linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
        
        linha_dois_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
        
        if(linha_dois.upper() == "CPF"):
            cpf.append(linha_dois_text)
            
        elif(linha_tres.upper() == "CPF"):
            cpf.append(linha_tres_text)
            
        else:
            cpf.append("--")
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        cns.append(i_cns[1])
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
        
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        try:
            linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
            linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
            if(linha_tres.upper() == 'PROFISSIONAL'):
                profissional.append(linha_tres_text)
        except:
            continue
        try:
            linha_quatro = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[1]').text
            linha_quatro_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
            if(linha_quatro.upper() == 'PROFISSIONAL'):
                profissional.append(linha_quatro_text)
        except:
            continue
        
        try:
            linha_cinco = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[1]').text
            linha_cinco_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[2]').text
            if(linha_cinco.upper() == 'PROFISSIONAL'):
                profissional.append(linha_cinco_text)
        except:
            continue
        
        try:
            linha_seis = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[1]').text
            linha_seis_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[2]').text
            if(linha_seis.upper() == 'PROFISSIONAL'):
                profissional.append(linha_seis_text)
            
        except:
            continue
            
        time.sleep(1)

    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')    
    return df_individual

# indi = get_indivivual_info()

In [14]:
children = get_individual_info()


Extração iniciada as: 2024-01-24 19:30:19.791863
Iniciando lista de links: 2024-01-24 19:30:36.586197


Traceback (most recent call last):
  File "/tmp/ipykernel_184813/2182560134.py", line 23, in get_table
    page.click()
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py", line 80, in click
    self._execute(Command.CLICK_ELEMENT)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py", line 633, in _execute
    return self._parent.execute(command, params)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.ElementClickInterceptedException: Message: Element <a class="page-link" href="#"> is not clickable at point (1200,665) because another element <li id="individuo-list-

acabaram as paginas
lista de links finalizada as: 2024-01-24 19:31:19.861720
Iniciando individuos : 2024-01-24 19:31:19.861744
Finalizando individuos : 2024-01-24 19:46:37.205891
Extração finalizada as: 2024-01-24 19:46:37.206566


In [15]:
# display(children)
# children.to_excel('teste.xlsx', index=False)
children = children[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'link']]

In [16]:
display(children)
children.to_excel(f'{data_e_hora}_{name}_children.xlsx', index=False)

,cpf,name,b_date,sex,race,m_name,link
0,18293990473,ALICE BEATRIZ ANGELO DA SILVA,22/09/2022,FEMININO,PARDA,STEFANE RAÍSSA FERREIRA DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
1,16772047451,ALLANA BEATRIZ PEREIRA DA SILVA,19/10/2019,FEMININO,PARDA,KAROLAINE DE BRITO PEREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
2,18605669446,ANA CECILIA LIMA NOGUEIRA,24/07/2023,FEMININO,BRANCA,PATRICIA RENISON LIMA NOGUEIRA,https://afogadosdaingazeira.ephealth.com.br/po...
3,16367155406,ANA CECILIA RIBEIRO PEREIRA,27/03/2019,FEMININO,BRANCA,SAMARA DA SILVA RIBEIRO,https://afogadosdaingazeira.ephealth.com.br/po...
4,17857876402,ANA LIZ BEZERRA DE MORAIS ALVES,29/09/2021,FEMININO,PARDA,PRISCILA BEZERRA DE MORAIS ALVES,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...
243,16792465429,VICTOR MANOEL DA SILVA FERREIRA,20/10/2019,MASCULINO,BRANCA,JACILENE AVELINA DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
244,18420754463,WEVERTON SAULO DA SILVA CANDIDO,20/01/2023,MASCULINO,PARDA,SARA FERNANDA DA SILVA CÂNDIDO,https://afogadosdaingazeira.ephealth.com.br/po...
245,18524047445,WITHALY GABRIELLY QUEIROZ SILVA,21/03/2023,FEMININO,PARDA,WITHALY NAIANE PEREIRA DA SILVA BATISTA,https://afogadosdaingazeira.ephealth.com.br/po...
246,17196079400,YASMIN MENDES BARBOSA,20/05/2020,FEMININO,PARDA,SIMARA MENDES DA SILVA VASCO,https://afogadosdaingazeira.ephealth.com.br/po...


In [17]:
sample = ["https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=9c3541f8-1bab-44db-9d88-9785392c3323",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=1f01706f-86ad-4cd7-b439-ed91d10f2406",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=10e797af-b61c-4fb6-8003-24d0eddc2ea7",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=ac61d929-496f-47e5-a280-2cf318eb42db",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=e05f43ae-f83a-41b9-a1c2-44192a01ae7d",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=38a88611-f158-47f9-8778-01d4a5a52cf3",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=a2843144-45a7-4634-b402-8418347c0012",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=77299017-0fdb-4e98-83ff-702e4a4a01be",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=1c7a74ae-db69-48c7-97d7-b84d5d39897b",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=ad6a93ca-9434-4ef0-be36-e5e112170dd1",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=5c0195e8-0d05-445f-aa52-6db283ea4a46",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=4fc2ea50-daaf-42ab-b76f-43b33ac19dbd",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=4dc2ec18-96be-4301-8d9f-500e2158c25b",
"https://afogadosdaingazeira.ephealth.com.br/portal/individuo?action=view&id=51bf1d1e-1991-4f0e-a8ea-e0df581b64d8"]
